In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.ticker import (
    AutoMinorLocator, 
    MultipleLocator,
)

def histogram_to_probabilities_of_deviation(hist, a=5.5, precision=0.01):
    res = np.zeros(hist.shape)
    total = sum(hist.T[0])    # equal to the number of iterations
    h = hist.shape[0]
    a = int(a / precision)
    up = h - a > a
    upper_bound = h - a if up else a
    
    for j, col in enumerate(hist.T):
        s = total - col[a]
        res[a][j] = 1.0
        for i in range(1, upper_bound):
            p = s / total
            u = a + i < h
            d = a - i >= 0
            s -= (col[a + i] if u else 0) + (col[a - i] if d else 0)
            if u:
                res[a + i][j] = p
            if d:
                res[a - i][j] = p
    
    return res


def setup_axis(
    ax, xlim, ylim,
    xlabel="days",
    ylabel="average",
    figsize=(40, 20),
    labels_font_size=24,
    ticks_font_size=20,
    x_major_ticks=100,
    x_minor_ticks=4,
    y_major_ticks=0.5,
    y_minor_ticks=5,
    major_line_width=1.00,
    minor_line_width=0.25,
    text_color="white",
):
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    
    ax.set_xlabel(xlabel, fontsize=labels_font_size, color=text_color)
    ax.set_ylabel(ylabel, fontsize=labels_font_size, color=text_color)
    
    ax.tick_params(axis="both", labelsize=ticks_font_size, labelcolor=text_color)

    ax.xaxis.set_major_locator(MultipleLocator(x_major_ticks))
    ax.xaxis.set_minor_locator(AutoMinorLocator(x_minor_ticks))
    ax.yaxis.set_major_locator(MultipleLocator(y_major_ticks))
    ax.yaxis.set_minor_locator(AutoMinorLocator(y_minor_ticks))
    
    ax.grid(b=True, which="major", axis="both", linestyle="-", linewidth=major_line_width)
    ax.grid(b=True, which="minor", axis="both", linestyle="-", linewidth=minor_line_width)
    

def setup_colorbar(fig, im, ax, text_color="white", label="probability, log10"):
    cb = fig.colorbar(im, ax=ax)
    cb.set_label(label, color=text_color, fontsize=24)
    cb.ax.yaxis.set_tick_params(color=text_color, labelsize=20)
    plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color=text_color)

    
def plot_data(ax, data, days, precision, cmap):
    return ax.pcolormesh(
        np.arange(0, days, 1),
        np.arange(0, 12 + precision, precision), 
        data, 
        vmin=data.min(), vmax=data.max(),
        cmap=cmap,
    #     norm=norm, 
        shading="auto",
    )

def plot_all(
    data, 
    averages,
    days,
    label,
    figsize = None,
    text_color = "white",
    cmap = "inferno",
    line_color = "white",
    line_width = 3,
    precision = 0.01, 
):
    figsize = (len(averages) / 10, 20) if figsize is None else figsize
    
    fig, ax = plt.subplots(figsize=figsize)
    fig.suptitle(label, color=text_color, fontsize=24)

    cmap=plt.get_cmap(cmap)
    im = plot_data(
        data=data, 
        ax=ax,
        days=days, precision=precision, cmap=cmap,
    )
    ax.plot(
        np.arange(0, len(averages), 1),
        averages,
        color=line_color,
        linewidth=line_width,
    )
    setup_axis(ax, xlim, ylim)
    setup_colorbar(
        fig=fig,
        im=im,
        ax=ax,
        text_color=text_color,
    )
    
    return fig

In [ ]:
from datetime import datetime
from pathlib import Path

from matplotlib.ticker import MaxNLocator
from matplotlib.colors import BoundaryNorm

from common.common import (
    read_numbers,
    numbers_to_averages,
    create_histogram,
    normalize_histogram,
    log_of,
)


NUMBERS = read_numbers("data.txt")
AVERAGES = numbers_to_averages(NUMBERS)


DAYS = len(NUMBERS) + 25
ITERS = 10_000_000
BALLS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# NBINS = 15
PRECISION = 0.0005
SAVE_FILES=True


hist = create_histogram(days=DAYS, iterations=ITERS, balls=BALLS, precision=PRECISION)
normalized_hist = normalize_histogram(hist)
log_hist = log_of(normalized_hist)

probs = histogram_to_probabilities_of_deviation(hist, precision=PRECISION)
log_probs = log_of(probs)

xlim=[0, DAYS]
ylim=[4.0, 7.0]
text_color = "white"


cmap = "nipy_spectral"

cmap1 = cmap
line_color1 = "yellow"
fig1 = plot_all(
    data=log_hist,
    averages=AVERAGES,
    days=DAYS,
    label="probability of average, log10",
    precision=PRECISION,
    cmap=cmap1,
    line_color=line_color1,
    line_width=4,
)

plt.show()


cmap2 = cmap
line_color2 = "black"
fig2 = plot_all(
    data=log_probs,
    averages=AVERAGES,
    days=DAYS,
    label="probability of deviation, log10",
    precision=PRECISION,
    cmap=cmap2,
    line_color=line_color2,
    line_width=4,
)

plt.show()


if SAVE_FILES:
    image_path="./images"
    Path(image_path).mkdir(exist_ok=True)
    fig1.savefig(f"{image_path}/{datetime.now().date()}-hit_d{DAYS}_i{ITERS}_p{PRECISION}_{cmap1}-{line_color1}.png", facecolor="black")
    fig2.savefig(f"{image_path}/{datetime.now().date()}-dev_d{DAYS}_i{ITERS}_p{PRECISION}_{cmap2}-{line_color2}.png", facecolor="black")